# Group sticky notes by color

This notebook demonstrates how to move sticky notes into groups, based on color.

The sample mural for this notebook is here: [Sample mural](https://github.com/spackows/MURAL-API-Samples/blob/main/murals/sample-06_group.json)

You can create that mural using the notebook here: [Mural creating notebook](https://github.com/spackows/MURAL-API-Samples/blob/main/notebooks/sample-01_create-a-mural.ipynb)

The sample mural for this notebook (before moving the sticky notes) looks like the following image:

<img src="https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/images/sample-06_group.png" width="50%" title="Image of a mural" />

## Step 1: Collect the mural ID

You can find the mural ID in the url of a mural.

Mural urls look something like this:

```
https://app.mural.co/t/<workspace>/m/<workspace>/<id>/...
```

What you need to pass to the MURAL API is just after the `/m/`: the \<workspace> and the \<id>.  And you need to join then with a period.

For example, if you have a mural with this url:

```
https://app.mural.co/t/teamideas1234/m/teamideas1234/1234567890123/...
```

Then, the mural ID is: `teamideas1234.1234567890123`

In [38]:
g_mural_id = ""

## Step 2: Collect your OAuth token

In [39]:
g_auth_token = ""

## Step 3: Read the widgets from the mural

In this sample, some widgets in the mural are sticky notes and some are shapes - squares (rectangles, really).

In [40]:
import requests
import json

def listWidgets( mural_id, auth_token ):
    # https://developers.mural.co/public/reference/getmuralwidgets
    url = "https://app.mural.co/api/public/v1/murals/" + mural_id + "/widgets"
    headers = { "Accept": "application/json", "Authorization": "Bearer " + auth_token }
    response = requests.request( "GET", url, headers = headers )
    response_json = json.loads( response.text )
    msg = ""
    if "code" in response_json:
        msg += response_json["code"] + " "
    if "message" in response_json:
        msg += response_json["message"]
    if msg != "":
        print( msg )
        return None
    if "value" not in response_json:
        print( "No value returned" )
        return None
    return response_json["value"]

In [41]:
widgets_arr = listWidgets( g_mural_id, g_auth_token )

In [42]:
colors = { "#046307FF" : "green",
           "#FFA500FF" : "orange",
           "#0000CDFF" : "blue" }

sticky_groups = { "#046307FF" : [],
                  "#FFA500FF" : [],
                  "#0000CDFF" : [] }

for widget in widgets_arr:
    if "sticky note" == widget["type"]:
        color = widget["style"]["backgroundColor"]
        print( "[ " + colors[ color ].ljust(6) + " ]  " + widget["text"] )
        sticky_groups[ color ].append( widget )

[ green  ]  I love winter!
[ blue   ]  I don't do well in the heat
[ orange ]  Fall is colourful
[ green  ]  Spring makes me feel optimistic
[ blue   ]  I hate the cold
[ green  ]  Fall leaves are beautiful
[ green  ]  Nothing is as soothing as drinking hot chocolate after being out in the cold
[ green  ]  Swimming in the summer makes me so happy
[ blue   ]  I feel sad watching everything die off in the fall
[ green  ]  summer is for relaxing
[ orange ]  spring and fall are shorter
[ green  ]  seasons bring a change of pace
[ orange ]  We go south for the winter every year


In [43]:
boxes = { "#046307FF" : {}, # green
          "#FFA500FF" : {}, # orange
          "#0000CDFF" : {}, # blue
        }

for widget in widgets_arr:
    if "shape" == widget["type"]:
        boxes[ widget["style"]["strokeColor"] ] = { "x" : widget["x"], "y" : widget["y"], "width" : widget["width"], "height" : widget["height"] }

print( json.dumps( boxes, indent = 3 ) )

{
   "#046307FF": {
      "x": 90.07,
      "y": 524.32,
      "width": 731.9999420166015,
      "height": 790.5568181818182
   },
   "#FFA500FF": {
      "x": 923.74,
      "y": 523.91,
      "width": 731.9999420166015,
      "height": 791.3768718927558
   },
   "#0000CDFF": {
      "x": 1762.41,
      "y": 522.6,
      "width": 731.9999420166015,
      "height": 793.9968670099433
   }
}


## Step 4: Move sticky notes into the boxes by color

This method puts the sticky notes into the rectangles in columns.  

It's orderly with a bit of randomness to make it feel somewhat organic.

In [44]:
from random import randrange

def moveSticky( sticky_id, x, y, mural_id, auth_token ):
    url = "https://app.mural.co/api/public/v1/murals/" + mural_id + "/widgets/sticky-note/" + sticky_id
    headers = { "Content-Type" : "application/json", "Accept" : "vnd.mural.preview", "Authorization" : "Bearer " + auth_token }
    data = json.dumps( { "x" : x, "y" : y } )
    response = requests.request( "PATCH", url, headers=headers, data=data )
    response_json = json.loads( response.text )
    msg = ""
    if "code" in response_json:
        msg += response_json["code"] + " "
    if "message" in response_json:
        msg += response_json["message"]
    if msg != "":
        print( msg )

def lineUpInBox( box, stickies_arr, mural_id, auth_token ):
    box_padding = 0.05 * box["height"]
    top_edge = box["y"] + box_padding
    left_edge = box["x"] + box_padding
    right_edge = left_edge
    for sticky in stickies_arr:
        sticky_margin = 0.1 * sticky["width"]
        random_range = round( 0.2 * sticky["width"] )
        if ( top_edge + sticky["height"] ) > ( box["y"] + box["height"] - box_padding ):
            top_edge = box["y"] + box_padding
            left_edge = right_edge
        x = left_edge + randrange( random_range )
        if ( x + sticky["width"] ) > right_edge:
            right_edge = x + sticky["width"]
        y = top_edge + randrange( random_range ) + sticky_margin
        top_edge = y + sticky["height"]
        moveSticky( sticky["id"], x, y, mural_id, auth_token )

def lineUpInBoxes( boxes, sticky_groups, mural_id, auth_token ):
    for color in boxes.keys():
        box = boxes[ color ]
        stickies_arr = sticky_groups[ color ]
        lineUpInBox( box, stickies_arr, mural_id, auth_token )

In [26]:
import time
time.sleep(5)

# Quick!  After running this cell, switch to your browser 
# tab where the mural is to see the stickies move
# ...

lineUpInBoxes( boxes, sticky_groups, g_mural_id, g_auth_token )

Like this:

<img src="https://github.com/spackows/MURAL-API-Samples/blob/main/images/sample-06_group.gif?raw=true" width="50%" title="Sticky notes moving" />